In [ ]:
import random
import string
from tqdm import tqdm

In [9]:
import random
import string
from tqdm import tqdm
import os

# Define constants
MAX_LENGTH = 200
OOD_MAX_LENGTH = 400
VALID_CHARACTERS = ["s", "a", "b", "c", 'd', 'f', 'g', 'x', 'y', 'z', "e", "p"]
MAIN_CHARACTERS = ["a", "b", "c", 'd', 'f', 'g', 'x', 'y', 'z']
START_TOKEN = "s"
END_TOKEN = "e"
PADDING_TOKEN = "p"
VALID_RATIO = 0.5  # Half of the dataset should be valid a*b* strings


# Function to generate valid a*b* strings
def generate_valid_string(min_length=0, max_length=MAX_LENGTH):
    # Ensure the minimum length accounts for START and END tokens
    min_length = max(min_length, 2)
    
    while True:
        # Generate the string with 'a's and 'b's while ensuring no three consecutive 'a's
        valid_str = ""
        valid_max = random.randint(0, max_length - 2)
        while len(valid_str) < valid_max:
            next_char = random.choice(['a', 'b'])
            if next_char == 'a' and valid_str.endswith('aaa'):
                next_char = random.choice(['b'])
            valid_str += next_char
            
        
        # Check if the length of the valid_str is within the desired range
        if len(valid_str) <= max_length - 2:
            break
    
    # Calculate the padding needed
    # num_p = random.randint(0, max_length - len(valid_str) - 2)
    return (
        # PADDING_TOKEN * num_p +
        START_TOKEN
        + valid_str
        + END_TOKEN
        # + PADDING_TOKEN * ((max_length - len(valid_str) - 2) - num_p)
        + PADDING_TOKEN * ((max_length - len(valid_str) - 2))
    )



# Function to generate invalid strings
def generate_invalid_string(min_length=0, max_length=MAX_LENGTH):
    # Ensure the minimum length accounts for START and END tokens    
    while True:
        # Generate the base string with random 'a's and 'b's
        # num_invalids = int((max_length - min_length) * 0.05)
        num_invalids = 3


        base_str_length = random.randint(0, max_length - 2 - num_invalids*4)  # Account for START, END tokens, and at least three 'a's
        base_str = ''.join(random.choice(['a', 'b']) for _ in range(base_str_length))
        invalid_str = base_str
        
        # Insert 'aaa' at a random position
        for i in range(num_invalids):
            insert_pos = random.randint(0, len(invalid_str))
            string = random.choice(["aaaa"])
            invalid_str = invalid_str[:insert_pos] + string + invalid_str[insert_pos:]
            # if base_str_length % 2 == 0:
            #     insert_pos = random.randint(0, len(invalid_str))
            #     string = random.choice(["a"])
            #     invalid_str = invalid_str[:insert_pos] + string + invalid_str[insert_pos:]

        # Ensure the length of the valid_str is within the desired range
        if len(invalid_str) <= max_length - 2:
            break
    
    # Calculate the padding needed
    # num_p = random.randint(0, max_length - len(invalid_str) - 2)
    return (
        # PADDING_TOKEN * num_p + 
        START_TOKEN
        + invalid_str
        + END_TOKEN
        # + PADDING_TOKEN * ((max_length - len(invalid_str) - 2) - num_p)
        + PADDING_TOKEN * ((max_length - len(invalid_str) - 2))
    )


# Generate dataset
dataset = []
num_samples = 50000  # Total number of samples

for _ in tqdm(range(num_samples)):
    while True:
        x = generate_valid_string()
        if not (len(x) == MAX_LENGTH):
            continue
        x += PADDING_TOKEN * (OOD_MAX_LENGTH - MAX_LENGTH)
        #print(x)
        if not (len(x) == OOD_MAX_LENGTH):
            #print('here')
            continue
        if (x, 1) not in dataset:
            #print('Here2')
            dataset.append((x, 1))
            break


# remove duplicates
valid_dataset = list(set(dataset))
dataset = []

for _ in tqdm(range(num_samples)):
    while True:
        x = generate_invalid_string()
        if not (len(x) == MAX_LENGTH):
            continue
        x += PADDING_TOKEN * (OOD_MAX_LENGTH - MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 0) not in dataset:
            dataset.append((x, 0))
            break

# Remove all duplicates
invalid_dataset = list(set(dataset))[:len(valid_dataset)]
print(f'len(valid_dataset): {len(valid_dataset)}')
print(f'len(invalid_dataset): {len(invalid_dataset)}')

split = len(valid_dataset) * 4 // 5
train_dataset = valid_dataset[:split] + invalid_dataset[:split]
test_dataset = valid_dataset[split:] + invalid_dataset[split:]

num_ood_samples = 15000
dataset = []
for _ in tqdm(range(num_ood_samples)):
    while True:
        x = generate_valid_string(min_length=MAX_LENGTH + 2, max_length=OOD_MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 1) not in dataset:
            dataset.append((x, 1))
            break

# remove duplicates
ood_valid_dataset = list(set(dataset))
dataset = []

for _ in tqdm(range(num_ood_samples)):
    while True:
        x = generate_invalid_string(min_length=MAX_LENGTH + 2, max_length=OOD_MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 0) not in dataset:
            dataset.append((x, 0))
            break

# Remove all duplicates
ood_invalid_dataset = list(set(dataset))[:len(ood_valid_dataset)]
print('len(ood_valid_dataset): ', len(ood_valid_dataset), 'len(ood_invalid_dataset): ', len(ood_invalid_dataset))

ood_dataset = ood_valid_dataset + ood_invalid_dataset
print('len(train_dataset): ', len(train_dataset))
print('len(test_dataset): ',  len(test_dataset))
print('len(ood_dataset): ',  len(ood_dataset))

dir = f'/orion/u/yrichard/n/key_trials/test_aaaa_{MAX_LENGTH}_{OOD_MAX_LENGTH}'
os.makedirs(dir, exist_ok=True)

# Write to file
with open(dir + "/train_dataset.txt", "w") as f:
    for data, label in train_dataset:
        f.write(f"{data} {label}\n")

with open(dir + "/test_dataset.txt", "w") as f:
    for data, label in test_dataset:
        f.write(f"{data} {label}\n")

with open(dir + "/ood_dataset.txt", "w") as f:
    for data, label in ood_dataset:
        f.write(f"{data} {label}\n")

print(dir)

  0%|          | 0/50000 [00:00<?, ?it/s]

100%|██████████| 50000/50000 [00:40<00:00, 1225.52it/s]


len(valid_dataset): 50000
len(invalid_dataset): 50000


100%|██████████| 15000/15000 [00:04<00:00, 3103.12it/s]


len(ood_valid_dataset):  15000 len(ood_invalid_dataset):  15000
len(train_dataset):  80000
len(test_dataset):  20000
len(ood_dataset):  30000
/orion/u/yrichard/n/key_trials/test_aaaa_200_400


In [10]:
import random
import string
from tqdm import tqdm
import os

# Define constants
MAX_LENGTH = 200
OOD_MAX_LENGTH = 400
VALID_CHARACTERS = ["s", "a", "b", "c", 'd', 'f', 'g', 'x', 'y', 'z', "e", "p"]
MAIN_CHARACTERS = ["a", "b", "c", 'd', 'f', 'g', 'x', 'y', 'z']
START_TOKEN = "s"
END_TOKEN = "e"
PADDING_TOKEN = "p"
VALID_RATIO = 0.5  # Half of the dataset should be valid a*b* strings


# Function to generate valid a*b* strings
def generate_valid_string(min_length=0, max_length=MAX_LENGTH):
    # Ensure the minimum length accounts for START and END tokens
    min_length = max(min_length, 2)
    
    while True:
        # Generate the string with 'a's and 'b's while ensuring no three consecutive 'a's
        valid_str = ""
        valid_max = random.randint(0, max_length - 2)
        while len(valid_str) < valid_max:
            next_char = random.choice(['x', 'y', 'z'])
            # if next_char == 'c' and valid_str.endswith('ab'):
            #     next_char = random.choice(['a','b'])
            valid_str += next_char
            
        
        # Check if the length of the valid_str is within the desired range
        if len(valid_str) <= max_length - 2:
            break
    
    # Calculate the padding needed
    # num_p = random.randint(0, max_length - len(valid_str) - 2)
    return (
        # PADDING_TOKEN * num_p +
        START_TOKEN
        + valid_str
        + END_TOKEN
        # + PADDING_TOKEN * ((max_length - len(valid_str) - 2) - num_p)
        + PADDING_TOKEN * ((max_length - len(valid_str) - 2))
    )



# Function to generate invalid strings
def generate_invalid_string(min_length=0, max_length=MAX_LENGTH):
    # Ensure the minimum length accounts for START and END tokens    
    while True:
        # Generate the base string with random 'a's and 'b's
        # num_invalids = int((max_length - min_length) * 0.05)
        num_invalids = 3


        base_str_length = random.randint(0, max_length - 2 - num_invalids*1)  # Account for START, END tokens, and at least three 'a's
        base_str = ''.join(random.choice(['x', 'y', 'z']) for _ in range(base_str_length))
        invalid_str = base_str
        
        # Insert 'aaa' at a random position
        for i in range(num_invalids):
            insert_pos = random.randint(0, len(invalid_str))
            string = random.choice(["a", 'b', 'c'])
            invalid_str = invalid_str[:insert_pos] + string + invalid_str[insert_pos:]
            # if base_str_length % 2 == 0:
            #     insert_pos = random.randint(0, len(invalid_str))
            #     string = random.choice(["a"])
            #     invalid_str = invalid_str[:insert_pos] + string + invalid_str[insert_pos:]

        # Ensure the length of the valid_str is within the desired range
        if len(invalid_str) <= max_length - 2:
            break
    
    # Calculate the padding needed
    # num_p = random.randint(0, max_length - len(invalid_str) - 2)
    return (
        # PADDING_TOKEN * num_p + 
        START_TOKEN
        + invalid_str
        + END_TOKEN
        # + PADDING_TOKEN * ((max_length - len(invalid_str) - 2) - num_p)
        + PADDING_TOKEN * ((max_length - len(invalid_str) - 2))
    )


# Generate dataset
dataset = []
num_samples = 50000  # Total number of samples

for _ in tqdm(range(num_samples)):
    while True:
        x = generate_valid_string()
        if not (len(x) == MAX_LENGTH):
            continue
        x += PADDING_TOKEN * (OOD_MAX_LENGTH - MAX_LENGTH)
        #print(x)
        if not (len(x) == OOD_MAX_LENGTH):
            #print('here')
            continue
        if (x, 1) not in dataset:
            #print('Here2')
            dataset.append((x, 1))
            break


# remove duplicates
valid_dataset = list(set(dataset))
dataset = []

for _ in tqdm(range(num_samples)):
    while True:
        x = generate_invalid_string()
        if not (len(x) == MAX_LENGTH):
            continue
        x += PADDING_TOKEN * (OOD_MAX_LENGTH - MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 0) not in dataset:
            dataset.append((x, 0))
            break

# Remove all duplicates
invalid_dataset = list(set(dataset))[:len(valid_dataset)]
print(f'len(valid_dataset): {len(valid_dataset)}')
print(f'len(invalid_dataset): {len(invalid_dataset)}')

split = len(valid_dataset) * 4 // 5
train_dataset = valid_dataset[:split] + invalid_dataset[:split]
test_dataset = valid_dataset[split:] + invalid_dataset[split:]

num_ood_samples = 15000
dataset = []
for _ in tqdm(range(num_ood_samples)):
    while True:
        x = generate_valid_string(min_length=MAX_LENGTH + 2, max_length=OOD_MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 1) not in dataset:
            dataset.append((x, 1))
            break

# remove duplicates
ood_valid_dataset = list(set(dataset))
dataset = []

for _ in tqdm(range(num_ood_samples)):
    while True:
        x = generate_invalid_string(min_length=MAX_LENGTH + 2, max_length=OOD_MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 0) not in dataset:
            dataset.append((x, 0))
            break

# Remove all duplicates
ood_invalid_dataset = list(set(dataset))[:len(ood_valid_dataset)]
print('len(ood_valid_dataset): ', len(ood_valid_dataset), 'len(ood_invalid_dataset): ', len(ood_invalid_dataset))

ood_dataset = ood_valid_dataset + ood_invalid_dataset
print('len(train_dataset): ', len(train_dataset))
print('len(test_dataset): ',  len(test_dataset))
print('len(ood_dataset): ',  len(ood_dataset))

dir = f'/orion/u/yrichard/n/key_trials/test_NOa-b-c_{MAX_LENGTH}_{OOD_MAX_LENGTH}'
os.makedirs(dir, exist_ok=True)

# Write to file
with open(dir + "/train_dataset.txt", "w") as f:
    for data, label in train_dataset:
        f.write(f"{data} {label}\n")

with open(dir + "/test_dataset.txt", "w") as f:
    for data, label in test_dataset:
        f.write(f"{data} {label}\n")

with open(dir + "/ood_dataset.txt", "w") as f:
    for data, label in ood_dataset:
        f.write(f"{data} {label}\n")

print(dir)

  0%|          | 0/50000 [00:00<?, ?it/s]

100%|██████████| 50000/50000 [00:38<00:00, 1299.11it/s]


len(valid_dataset): 50000
len(invalid_dataset): 50000


100%|██████████| 15000/15000 [00:04<00:00, 3254.95it/s]


len(ood_valid_dataset):  15000 len(ood_invalid_dataset):  15000
len(train_dataset):  80000
len(test_dataset):  20000
len(ood_dataset):  30000
/orion/u/yrichard/n/key_trials/test_NOa-b-c_200_400


In [7]:
def generate_invalid_string(min_length=0, max_length=MAX_LENGTH):
    # Ensure the minimum length accounts for START and END tokens    
    while True:
        # Generate the base string with random 'a's and 'b's
        base_str_length = random.randint(0, max_length - 5)  # Account for START, END tokens, and at least three 'a's
        base_str = ''.join(random.choice(['a', 'b', 'c']) for _ in range(base_str_length))
        
        # Insert 'aaa' at a random position
        insert_pos = random.randint(0, len(base_str))
        valid_str = base_str[:insert_pos] + 'abc' + base_str[insert_pos:]
        
        # Ensure the length of the valid_str is within the desired range
        if min_length - 2 <= len(valid_str) <= max_length - 2:
            break
    
    # Calculate the padding needed
    num_p = random.randint(0, max_length - len(valid_str) - 2)
    return (
        PADDING_TOKEN * num_p
        + START_TOKEN
        + valid_str
        + END_TOKEN
        + PADDING_TOKEN * ((max_length - len(valid_str) - 2) - num_p)
    )

for i in range(5):
    print(generate_invalid_string(10, 20))

psacbacbccbbaabccaae
pscabcbbbabceppppppp
sbccacaccaaaabcbccep
pppppppsabbabccbcbce
psbcbacbcabcabbaaaep


In [11]:
import random
import string
from tqdm import tqdm

# Define constants
MAX_LENGTH = 200
OOD_MAX_LENGTH = 400
VALID_CHARACTERS = ["s", "a", "b", "c", "e", "p"]
MAIN_CHARACTERS = ["a", "b", "c"]
START_TOKEN = "s"
END_TOKEN = "e"
PADDING_TOKEN = "p"
VALID_RATIO = 0.5  # Half of the dataset should be valid a*b* strings


# Function to generate valid a*b* strings
def generate_valid_string(min_length=0, max_length=MAX_LENGTH):
    # Ensure the minimum length accounts for START and END tokens
    min_length = max(min_length, 2)
    
    while True:
        # Generate the string with 'a's and 'b's while ensuring no three consecutive 'a's
        valid_str = ""
        valid_max = random.randint(0, max_length - 2)
        while len(valid_str) < valid_max:
            next_char = random.choice(['a', 'b', 'c'])
            if next_char == 'c' and valid_str.endswith('ab'):
                next_char = random.choice(['a', 'b'])
            valid_str += next_char
            
        
        # Check if the length of the valid_str is within the desired range
        if len(valid_str) <= max_length - 2:
            break
    
    # Calculate the padding needed
    num_p = random.randint(0, max_length - len(valid_str) - 2)
    return (
        PADDING_TOKEN * num_p
        + START_TOKEN
        + valid_str
        + END_TOKEN
        + PADDING_TOKEN * ((max_length - len(valid_str) - 2) - num_p)
    )



# Function to generate invalid strings
def generate_invalid_string(min_length=0, max_length=MAX_LENGTH):
    # Ensure the minimum length accounts for START and END tokens    
    while True:
        # Generate the base string with random 'a's and 'b's
        base_str_length = random.randint(0, max_length - 5)  # Account for START, END tokens, and at least three 'a's
        base_str = ''.join(random.choice(['a', 'b', 'c']) for _ in range(base_str_length))
        
        # Insert 'aaa' at a random position
        insert_pos = random.randint(0, len(base_str))
        valid_str = base_str[:insert_pos] + 'abc' + base_str[insert_pos:]
        
        # Ensure the length of the valid_str is within the desired range
        if len(valid_str) <= max_length - 2:
            break
    
    # Calculate the padding needed
    num_p = random.randint(0, max_length - len(valid_str) - 2)
    return (
        PADDING_TOKEN * num_p
        + START_TOKEN
        + valid_str
        + END_TOKEN
        + PADDING_TOKEN * ((max_length - len(valid_str) - 2) - num_p)
    )


# Generate dataset
dataset = []
num_samples = 15000  # Total number of samples

for _ in tqdm(range(num_samples)):
    while True:
        x = generate_valid_string()
        if not (len(x) == MAX_LENGTH):
            continue
        x += PADDING_TOKEN * (OOD_MAX_LENGTH - MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 1) not in dataset:
            dataset.append((x, 1))
            break


# remove duplicates
valid_dataset = list(set(dataset))
dataset = []

for _ in tqdm(range(num_samples)):
    while True:
        x = generate_invalid_string()
        if not (len(x) == MAX_LENGTH):
            continue
        x += PADDING_TOKEN * (OOD_MAX_LENGTH - MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 0) not in dataset:
            dataset.append((x, 0))
            break

# Remove all duplicates
invalid_dataset = list(set(dataset))[:len(valid_dataset)]
print(f'len(valid_dataset): {len(valid_dataset)}')
print(f'len(invalid_dataset): {len(invalid_dataset)}')

split = len(valid_dataset) * 4 // 5
train_dataset = valid_dataset[:split] + invalid_dataset[:split]
test_dataset = valid_dataset[split:] + invalid_dataset[split:]

num_ood_samples = 15000
dataset = []
for _ in tqdm(range(num_ood_samples)):
    while True:
        x = generate_valid_string(min_length=MAX_LENGTH + 2, max_length=OOD_MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 1) not in dataset:
            dataset.append((x, 1))
            break

# remove duplicates
ood_valid_dataset = list(set(dataset))
dataset = []

for _ in tqdm(range(num_ood_samples)):
    while True:
        x = generate_invalid_string(min_length=MAX_LENGTH + 2, max_length=OOD_MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 0) not in dataset:
            dataset.append((x, 0))
            break

# Remove all duplicates
ood_invalid_dataset = list(set(dataset))[:len(ood_valid_dataset)]
print('len(ood_valid_dataset): ', len(ood_valid_dataset), 'len(ood_invalid_dataset): ', len(ood_invalid_dataset))

ood_dataset = ood_valid_dataset + ood_invalid_dataset
print('len(train_dataset): ', len(train_dataset))
print('len(test_dataset): ',  len(test_dataset))
print('len(ood_dataset): ',  len(ood_dataset))

dir = '/orion/u/yrichard/n/key_trials/test_abc_200_400_padded'


# Write to file
with open(dir + "/train_dataset.txt", "w") as f:
    for data, label in train_dataset:
        f.write(f"{data} {label}\n")

with open(dir + "/test_dataset.txt", "w") as f:
    for data, label in test_dataset:
        f.write(f"{data} {label}\n")

with open(dir + "/ood_dataset.txt", "w") as f:
    for data, label in ood_dataset:
        f.write(f"{data} {label}\n")

100%|██████████| 15000/15000 [00:10<00:00, 1474.99it/s]


len(valid_dataset): 15000
len(invalid_dataset): 15000


100%|██████████| 15000/15000 [00:11<00:00, 1272.53it/s]


len(ood_valid_dataset):  15000 len(ood_invalid_dataset):  15000
len(train_dataset):  24000
len(test_dataset):  6000
len(ood_dataset):  30000


# Non-padded

In [1]:
import random
import string
from tqdm import tqdm

# Define constants
MAX_LENGTH = 200
OOD_MAX_LENGTH = 400
VALID_CHARACTERS = ["s", "a", "b", "c", "e", "p"]
MAIN_CHARACTERS = ["a", "b", "c"]
START_TOKEN = "s"
END_TOKEN = "e"
PADDING_TOKEN = "p"
VALID_RATIO = 0.5  # Half of the dataset should be valid a*b* strings


# Function to generate valid a*b* strings
def generate_valid_string(min_length=0, max_length=MAX_LENGTH):
    # Ensure the minimum length accounts for START and END tokens
    min_length = max(min_length, 2)
    
    while True:
        # Generate the string with 'a's and 'b's while ensuring no three consecutive 'a's
        valid_str = ""
        valid_max = random.randint(0, max_length - 2)
        while len(valid_str) < valid_max:
            next_char = random.choice(['a', 'b', 'c'])
            if next_char == 'c' and valid_str.endswith('ab'):
                next_char = random.choice(['a', 'b'])
            valid_str += next_char
            
        
        # Check if the length of the valid_str is within the desired range
        if len(valid_str) <= max_length - 2:
            break
    
    # Calculate the padding needed
    # num_p = random.randint(0, max_length - len(valid_str) - 2)
    return (
        #PADDING_TOKEN * num_p
        START_TOKEN
        + valid_str
        + END_TOKEN
        + PADDING_TOKEN * ((max_length - len(valid_str) - 2))
    )



# Function to generate invalid strings
def generate_invalid_string(min_length=0, max_length=MAX_LENGTH):
    # Ensure the minimum length accounts for START and END tokens    
    while True:
        # Generate the base string with random 'a's and 'b's
        base_str_length = random.randint(0, max_length - 5)  # Account for START, END tokens, and at least three 'a's
        base_str = ''.join(random.choice(['a', 'b', 'c']) for _ in range(base_str_length))
        
        # Insert 'aaa' at a random position
        insert_pos = random.randint(0, len(base_str))
        valid_str = base_str[:insert_pos] + 'abc' + base_str[insert_pos:]
        
        # Ensure the length of the valid_str is within the desired range
        if len(valid_str) <= max_length - 2:
            break
    
    # Calculate the padding needed
    # num_p = random.randint(0, max_length - len(valid_str) - 2)
    return (
        START_TOKEN
        + valid_str
        + END_TOKEN
        + PADDING_TOKEN * ((max_length - len(valid_str) - 2))
    )


# Generate dataset
dataset = []
num_samples = 15000  # Total number of samples

for _ in tqdm(range(num_samples)):
    while True:
        x = generate_valid_string()
        if not (len(x) == MAX_LENGTH):
            continue
        x += PADDING_TOKEN * (OOD_MAX_LENGTH - MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 1) not in dataset:
            dataset.append((x, 1))
            break


# remove duplicates
valid_dataset = list(set(dataset))
dataset = []

for _ in tqdm(range(num_samples)):
    while True:
        x = generate_invalid_string()
        if not (len(x) == MAX_LENGTH):
            continue
        x += PADDING_TOKEN * (OOD_MAX_LENGTH - MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 0) not in dataset:
            dataset.append((x, 0))
            break

# Remove all duplicates
invalid_dataset = list(set(dataset))[:len(valid_dataset)]
print(f'len(valid_dataset): {len(valid_dataset)}')
print(f'len(invalid_dataset): {len(invalid_dataset)}')

split = len(valid_dataset) * 4 // 5
train_dataset = valid_dataset[:split] + invalid_dataset[:split]
test_dataset = valid_dataset[split:] + invalid_dataset[split:]

num_ood_samples = 15000
dataset = []
for _ in tqdm(range(num_ood_samples)):
    while True:
        x = generate_valid_string(min_length=MAX_LENGTH + 2, max_length=OOD_MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 1) not in dataset:
            dataset.append((x, 1))
            break

# remove duplicates
ood_valid_dataset = list(set(dataset))
dataset = []

for _ in tqdm(range(num_ood_samples)):
    while True:
        x = generate_invalid_string(min_length=MAX_LENGTH + 2, max_length=OOD_MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 0) not in dataset:
            dataset.append((x, 0))
            break

# Remove all duplicates
ood_invalid_dataset = list(set(dataset))[:len(ood_valid_dataset)]
print('len(ood_valid_dataset): ', len(ood_valid_dataset), 'len(ood_invalid_dataset): ', len(ood_invalid_dataset))

ood_dataset = ood_valid_dataset + ood_invalid_dataset
print('len(train_dataset): ', len(train_dataset))
print('len(test_dataset): ',  len(test_dataset))
print('len(ood_dataset): ',  len(ood_dataset))

dir = '/orion/u/yrichard/n/key_trials/test_abc_200_400'


# Write to file
with open(dir + "/train_dataset.txt", "w") as f:
    for data, label in train_dataset:
        f.write(f"{data} {label}\n")

with open(dir + "/test_dataset.txt", "w") as f:
    for data, label in test_dataset:
        f.write(f"{data} {label}\n")

with open(dir + "/ood_dataset.txt", "w") as f:
    for data, label in ood_dataset:
        f.write(f"{data} {label}\n")

100%|██████████| 15000/15000 [00:07<00:00, 1949.15it/s]


len(valid_dataset): 15000
len(invalid_dataset): 15000


100%|██████████| 15000/15000 [00:08<00:00, 1797.48it/s]


len(ood_valid_dataset):  15000 len(ood_invalid_dataset):  15000
len(train_dataset):  24000
len(test_dataset):  6000
len(ood_dataset):  30000


In [19]:
def generate_invalid_string(min_length=0, max_length=MAX_LENGTH):
    # Ensure the minimum length accounts for START and END tokens    
    while True:
        # Generate the base string with random 'a's and 'b's
        base_str_length = max_length - 6  # Account for START, END tokens, and at least three 'a's
        base_str = ''.join(random.choice(['a', 'b']) for _ in range(base_str_length))
        
        # Insert 'aaa' at a random position
        insert_pos = random.randint(0, len(base_str))
        valid_str = base_str[:insert_pos] + 'aaaa' + base_str[insert_pos:]
        
        # Ensure the length of the valid_str is within the desired range
        if min_length - 2 <= len(valid_str) <= max_length - 2:
            break
    
    # Calculate the padding needed
    num_p = random.randint(0, max_length - len(valid_str) - 2)
    return (
        PADDING_TOKEN * num_p
        + START_TOKEN
        + valid_str
        + END_TOKEN
        + PADDING_TOKEN * ((max_length - len(valid_str) - 2) - num_p)
    )

for i in range(5):
    print(generate_invalid_string(min_length=10, max_length=20))


sbbabbaabbabbaaaaaae
saaaaaaaaaaaababbbae
saaaabbbbaabbabbaaae
sbaabbaaaaaaaaabbabe
sbbabaaaaaaabbbaaabe


In [10]:
import random
import string
from tqdm import tqdm

# Define constants
MAX_LENGTH = 200
OOD_MAX_LENGTH = 400
VALID_CHARACTERS = ["s", "a", "b", "e", "p"]
MAIN_CHARACTERS = ["a", "b"]
START_TOKEN = "s"
END_TOKEN = "e"
PADDING_TOKEN = "p"
VALID_RATIO = 0.5  # Half of the dataset should be valid a*b* strings


# Function to generate valid a*b* strings
def generate_valid_string(min_length=0, max_length=MAX_LENGTH):
    # Ensure the minimum length accounts for START and END tokens
    min_length = max(min_length, 2)
    
    while True:
        # Generate the string with 'a's and 'b's while ensuring no three consecutive 'a's
        valid_str = ""
        valid_length = random.randint(0, max_length - 2)
        while len(valid_str) < valid_length:
            next_char = random.choice(['a', 'b'])
            if next_char == 'a' and valid_str.endswith('aaa'):
                next_char = 'b'
            valid_str += next_char
            
        
        # Check if the length of the valid_str is within the desired range
        if len(valid_str) <= max_length - 2:
            break
    
    # Calculate the padding needed
    num_p = random.randint(0, max_length - len(valid_str) - 2)
    return (
        PADDING_TOKEN * num_p
        + START_TOKEN
        + valid_str
        + END_TOKEN
        + PADDING_TOKEN * ((max_length - len(valid_str) - 2) - num_p)
    )


# Function to generate invalid strings
def generate_invalid_string(min_length=0, max_length=MAX_LENGTH):
    # Ensure the minimum length accounts for START and END tokens    
    while True:
        # Generate the base string with random 'a's and 'b's
        base_str_length = random.randint(0, max_length - 6)  # Account for START, END tokens, and at least three 'a's
        base_str = ''.join(random.choice(['a', 'b']) for _ in range(base_str_length))
        
        # Insert 'aaa' at a random position
        insert_pos = random.randint(0, len(base_str))
        valid_str = base_str[:insert_pos] + 'aaaa' + base_str[insert_pos:]
        
        # Ensure the length of the valid_str is within the desired range
        if len(valid_str) <= max_length - 2:
            break
    
    # Calculate the padding needed
    num_p = random.randint(0, max_length - len(valid_str) - 2)
    return (
        PADDING_TOKEN * num_p
        + START_TOKEN
        + valid_str
        + END_TOKEN
        + PADDING_TOKEN * ((max_length - len(valid_str) - 2) - num_p)
    )


# Generate dataset
dataset = []
num_samples = 15000  # Total number of samples

for _ in tqdm(range(num_samples)):
    while True:
        x = generate_valid_string()
        if not (len(x) == MAX_LENGTH):
            continue
        x += PADDING_TOKEN * (OOD_MAX_LENGTH - MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 1) not in dataset:
            dataset.append((x, 1))
            break


# remove duplicates
valid_dataset = list(set(dataset))
dataset = []

for _ in tqdm(range(num_samples)):
    while True:
        x = generate_invalid_string()
        if not (len(x) == MAX_LENGTH):
            continue
        x += PADDING_TOKEN * (OOD_MAX_LENGTH - MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 0) not in dataset:
            dataset.append((x, 0))
            break

# Remove all duplicates
invalid_dataset = list(set(dataset))[:len(valid_dataset)]
print(f'len(valid_dataset): {len(valid_dataset)}')
print(f'len(invalid_dataset): {len(invalid_dataset)}')

split = len(valid_dataset) * 4 // 5
train_dataset = valid_dataset[:split] + invalid_dataset[:split]
test_dataset = valid_dataset[split:] + invalid_dataset[split:]

num_ood_samples = 15000
dataset = []
for _ in tqdm(range(num_ood_samples)):
    while True:
        x = generate_valid_string(min_length=MAX_LENGTH + 2, max_length=OOD_MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 1) not in dataset:
            dataset.append((x, 1))
            break

# remove duplicates
ood_valid_dataset = list(set(dataset))
dataset = []

for _ in tqdm(range(num_ood_samples)):
    while True:
        x = generate_invalid_string(min_length=MAX_LENGTH + 2, max_length=OOD_MAX_LENGTH)
        if not (len(x) == OOD_MAX_LENGTH):
            continue
        if (x, 0) not in dataset:
            dataset.append((x, 0))
            break

# Remove all duplicates
ood_invalid_dataset = list(set(dataset))[:len(ood_valid_dataset)]
print('len(ood_valid_dataset): ', len(ood_valid_dataset), 'len(ood_invalid_dataset): ', len(ood_invalid_dataset))

ood_dataset = ood_valid_dataset + ood_invalid_dataset
print('len(train_dataset): ', len(train_dataset))
print('len(test_dataset): ',  len(test_dataset))
print('len(ood_dataset): ',  len(ood_dataset))

dir = '/orion/u/yrichard/n/key_trials/test_aaaa_200_400_padded'


# Write to file
with open(dir + "/train_dataset.txt", "w") as f:
    for data, label in train_dataset:
        f.write(f"{data} {label}\n")

with open(dir + "/test_dataset.txt", "w") as f:
    for data, label in test_dataset:
        f.write(f"{data} {label}\n")

with open(dir + "/ood_dataset.txt", "w") as f:
    for data, label in ood_dataset:
        f.write(f"{data} {label}\n")

  0%|          | 0/15000 [00:00<?, ?it/s]

100%|██████████| 15000/15000 [00:07<00:00, 1875.87it/s]


len(valid_dataset): 15000
len(invalid_dataset): 15000


100%|██████████| 15000/15000 [00:12<00:00, 1230.45it/s]


len(ood_valid_dataset):  15000 len(ood_invalid_dataset):  15000
len(train_dataset):  24000
len(test_dataset):  6000
len(ood_dataset):  30000


In [9]:
dir = '/orion/u/yrichard/n/key_trials/test_aaa_200_400'


# Write to file
with open(dir + "/train_dataset.txt", "w") as f:
    for data, label in train_dataset:
        f.write(f"{data} {label}\n")

with open(dir + "/test_dataset.txt", "w") as f:
    for data, label in test_dataset:
        f.write(f"{data} {label}\n")

with open(dir + "/ood_dataset.txt", "w") as f:
    for data, label in ood_dataset:
        f.write(f"{data} {label}\n")